<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/convertiblebonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest

def flat_rate(today, forward, day_counter):
    return ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(forward)), day_counter)

def flat_vol(today, vol, day_counter):
    return ql.BlackConstantVol(today, ql.NullCalendar(), ql.QuoteHandle(ql.SimpleQuote(vol)), day_counter)

class ConvertibleBondTests(unittest.TestCase):

    def setUp(self):
        """Set up common variables for the tests, similar to CommonVars struct."""
        self.calendar = ql.TARGET()
        self.today = ql.Settings.instance().evaluationDate

        self.day_counter = ql.Actual360()
        self.frequency = ql.Annual
        self.settlement_days = 3
        self.fixing_days_initial_estimate = 2 # Used for initial issue date estimation

        # Initial setup for issue and maturity, then refine issueDate
        issue_date_temp = self.calendar.advance(self.today, self.fixing_days_initial_estimate, ql.Days)
        self.maturity_date = self.calendar.advance(issue_date_temp, 10, ql.Years)
        self.issue_date = self.calendar.advance(self.maturity_date, -10, ql.Years) # Ensure 10 year exact tenor
        self.fixing_days = self.calendar.businessDaysBetween(self.today, self.issue_date)

        self.underlying_quote = ql.SimpleQuote(50.0)
        self.dividend_yield_quote = ql.SimpleQuote(0.02)
        self.risk_free_rate_quote = ql.SimpleQuote(0.05)
        self.volatility_quote = ql.SimpleQuote(0.15)
        self.credit_spread_quote = ql.SimpleQuote(0.005)

        self.underlying_handle = ql.RelinkableQuoteHandle(self.underlying_quote)
        self.dividend_yield_handle = ql.RelinkableYieldTermStructureHandle(
            flat_rate(self.today, self.dividend_yield_quote.value(), self.day_counter)
        )
        self.risk_free_rate_handle = ql.RelinkableYieldTermStructureHandle(
            flat_rate(self.today, self.risk_free_rate_quote.value(), self.day_counter)
        )
        self.volatility_handle = ql.RelinkableBlackVolTermStructureHandle(
            flat_vol(self.today, self.volatility_quote.value(), self.day_counter)
        )
        self.credit_spread_handle = ql.RelinkableQuoteHandle(self.credit_spread_quote)

        # Update handles if quotes change
        self.dividend_yield_handle.linkTo(flat_rate(self.today, self.dividend_yield_quote.value(), self.day_counter))
        self.risk_free_rate_handle.linkTo(flat_rate(self.today, self.risk_free_rate_quote.value(), self.day_counter))
        self.volatility_handle.linkTo(flat_vol(self.today, self.volatility_quote.value(), self.day_counter))


        self.process = ql.BlackScholesMertonProcess(
            self.underlying_handle,
            self.dividend_yield_handle,
            self.risk_free_rate_handle,
            self.volatility_handle
        )

        self.no_callability = ql.CallabilitySchedule() # Empty schedule

        self.face_amount = 100.0
        self.redemption = 100.0
        # This will be updated in tests if needed
        self.conversion_ratio_default = self.redemption / self.underlying_quote.value()


    def test_bond(self):
        """
        When deeply out-of-the-money, the value of the convertible bond
        should equal that of the underlying plain-vanilla bond.
        """
        print("Testing out-of-the-money convertible bonds against vanilla bonds...")

        # Set conversion ratio to be deeply out-of-the-money
        conversion_ratio_ootm = 1.0e-16

        eu_exercise = ql.EuropeanExercise(self.maturity_date)
        am_exercise = ql.AmericanExercise(self.issue_date, self.maturity_date)

        time_steps = 1001
        # Make sure to use the OOTM conversion ratio for the test setup
        # (though it doesn't directly impact the engine setup for price-only)

        convertible_engine = ql.BinomialConvertibleEngineCRRAmerican(
            self.process, time_steps, self.credit_spread_handle
        )
        # Note: C++ used generic BinomialConvertibleEngine<CoxRossRubinstein>.
        # In Python, CRR is often BinomialCRRVanillaEngine or BinomialConvertibleEngineCRRAmerican.
        # The specific engine might matter for subtle differences.
        # If results diverge significantly, might need to check specific CRR engine bindings.
        # For now, assuming BinomialConvertibleEngineCRRAmerican is a reasonable choice.
        # A more generic form would be:
        # tree = ql.CoxRossRubinstein(self.process.time(self.maturity_date), time_steps, ...)
        # convertible_engine = ql.BinomialPricingEngine(tree, self.process) but this is for vanilla options
        # For convertibles, specific convertible engines are generally used.
        # Let's use the one from the constructor:
        # BinomialConvertibleEngine[CoxRossRubinstein](process, timeSteps, creditSpread) ->
        # ql.BinomialConvertibleEngine(self.process, "CoxRossRubinstein", time_steps, self.credit_spread_handle)
        # The above is not directly available.
        # Let's try the specific American CRR engine as it's common for convertibles
        # or use the generic ql.BinomialConvertibleEngine and specify tree type if available.
        # A common pattern for CRR in Python for convertibles:
        convertible_engine_CRR_generic = ql.BinomialConvertibleEngine(self.process, "CoxRossRubinstein", time_steps, self.credit_spread_handle)


        discount_curve = ql.YieldTermStructureHandle(
            ql.ForwardSpreadedTermStructure(self.risk_free_rate_handle, self.credit_spread_handle)
        )
        bond_engine = ql.DiscountingBondEngine(discount_curve)

        # --- Zero-coupon ---
        schedule_zero = ql.MakeSchedule().from_(self.issue_date).to(self.maturity_date) \
                                     .withFrequency(ql.Once) \
                                     .withCalendar(self.calendar) \
                                     .backwards().makeSchedule()

        eu_zero_convertible = ql.ConvertibleZeroCouponBond(
            eu_exercise, conversion_ratio_ootm, self.no_callability,
            self.issue_date, self.settlement_days, self.day_counter,
            schedule_zero, self.redemption
        )
        eu_zero_convertible.setPricingEngine(convertible_engine_CRR_generic)

        am_zero_convertible = ql.ConvertibleZeroCouponBond(
            am_exercise, conversion_ratio_ootm, self.no_callability,
            self.issue_date, self.settlement_days, self.day_counter,
            schedule_zero, self.redemption
        )
        am_zero_convertible.setPricingEngine(convertible_engine_CRR_generic)

        vanilla_zero_bond = ql.ZeroCouponBond(
            self.settlement_days, self.calendar, 100.0, self.maturity_date,
            ql.Following, self.redemption, self.issue_date
        )
        vanilla_zero_bond.setPricingEngine(bond_engine)

        tolerance_zero = 1.0e-2 * (self.face_amount / 100.0)

        error_eu_zero = abs(eu_zero_convertible.NPV() - vanilla_zero_bond.settlementValue())
        self.assertLessEqual(error_eu_zero, tolerance_zero,
                             f"Failed to reproduce EU zero-coupon bond price:\n"
                             f"    calculated: {eu_zero_convertible.NPV()}\n"
                             f"    expected:   {vanilla_zero_bond.settlementValue()}\n"
                             f"    error:      {error_eu_zero}")

        error_am_zero = abs(am_zero_convertible.NPV() - vanilla_zero_bond.settlementValue())
        self.assertLessEqual(error_am_zero, tolerance_zero,
                             f"Failed to reproduce AM zero-coupon bond price:\n"
                             f"    calculated: {am_zero_convertible.NPV()}\n"
                             f"    expected:   {vanilla_zero_bond.settlementValue()}\n"
                             f"    error:      {error_am_zero}")

        # --- Fixed-coupon ---
        coupons_fixed = [0.05]
        schedule_fixed = ql.MakeSchedule().from_(self.issue_date).to(self.maturity_date) \
                                      .withFrequency(self.frequency) \
                                      .withCalendar(self.calendar) \
                                      .backwards().makeSchedule()

        eu_fixed_convertible = ql.ConvertibleFixedCouponBond(
            eu_exercise, conversion_ratio_ootm, self.no_callability,
            self.issue_date, self.settlement_days, coupons_fixed, self.day_counter,
            schedule_fixed, self.redemption
        )
        eu_fixed_convertible.setPricingEngine(convertible_engine_CRR_generic)

        am_fixed_convertible = ql.ConvertibleFixedCouponBond(
            am_exercise, conversion_ratio_ootm, self.no_callability,
            self.issue_date, self.settlement_days, coupons_fixed, self.day_counter,
            schedule_fixed, self.redemption
        )
        am_fixed_convertible.setPricingEngine(convertible_engine_CRR_generic)

        vanilla_fixed_bond = ql.FixedRateBond(
            self.settlement_days, self.face_amount, schedule_fixed,
            coupons_fixed, self.day_counter, ql.Following,
            self.redemption, self.issue_date
        )
        vanilla_fixed_bond.setPricingEngine(bond_engine)

        tolerance_fixed = 2.0e-2 * (self.face_amount / 100.0)

        error_eu_fixed = abs(eu_fixed_convertible.NPV() - vanilla_fixed_bond.settlementValue())
        self.assertLessEqual(error_eu_fixed, tolerance_fixed,
                             f"Failed to reproduce EU fixed-coupon bond price:\n"
                             f"    calculated: {eu_fixed_convertible.NPV()}\n"
                             f"    expected:   {vanilla_fixed_bond.settlementValue()}\n"
                             f"    error:      {error_eu_fixed}")

        error_am_fixed = abs(am_fixed_convertible.NPV() - vanilla_fixed_bond.settlementValue())
        self.assertLessEqual(error_am_fixed, tolerance_fixed,
                             f"Failed to reproduce AM fixed-coupon bond price:\n"
                             f"    calculated: {am_fixed_convertible.NPV()}\n"
                             f"    expected:   {vanilla_fixed_bond.settlementValue()}\n"
                             f"    error:      {error_am_fixed}")

        # --- Floating-rate ---
        index_euribor = ql.Euribor1Y(discount_curve) # Link to the spreaded curve
        gearings_float = [1.0]
        spreads_float = [] # Empty in C++

        # Using same schedule as fixed for simplicity, as per C++ test
        schedule_float = schedule_fixed

        eu_floating_convertible = ql.ConvertibleFloatingRateBond(
            eu_exercise, conversion_ratio_ootm, self.no_callability,
            self.issue_date, self.settlement_days, index_euribor, self.fixing_days,
            spreads_float, self.day_counter, schedule_float, self.redemption
        )
        eu_floating_convertible.setPricingEngine(convertible_engine_CRR_generic)

        am_floating_convertible = ql.ConvertibleFloatingRateBond(
            am_exercise, conversion_ratio_ootm, self.no_callability,
            self.issue_date, self.settlement_days, index_euribor, self.fixing_days,
            spreads_float, self.day_counter, schedule_float, self.redemption
        )
        am_floating_convertible.setPricingEngine(convertible_engine_CRR_generic)

        # The C++ `floatSchedule` used a slightly different construction, but results in the same dates for Annual.
        # We'll use the same schedule_float for consistency with the convertible version.
        vanilla_floating_bond = ql.FloatingRateBond(
            self.settlement_days, self.face_amount, schedule_float,
            index_euribor, self.day_counter, ql.Following, self.fixing_days,
            gearings_float, spreads_float, [], [], # caps, floors
            False, # inArrears
            self.redemption, self.issue_date
        )

        # Coupon pricer setup
        # Empty OptionletVolatilityStructure handle as per C++
        optionlet_vol_handle = ql.OptionletVolatilityStructureHandle()
        pricer = ql.BlackIborCouponPricer(optionlet_vol_handle)
        ql.setCouponPricer(vanilla_floating_bond.cashflows(), pricer)

        vanilla_floating_bond.setPricingEngine(bond_engine)


        tolerance_float = 2.0e-2 * (self.face_amount / 100.0)

        error_eu_float = abs(eu_floating_convertible.NPV() - vanilla_floating_bond.settlementValue())
        self.assertLessEqual(error_eu_float, tolerance_float,
                             f"Failed to reproduce EU floating-rate bond price:\n"
                             f"    calculated: {eu_floating_convertible.NPV()}\n"
                             f"    expected:   {vanilla_floating_bond.settlementValue()}\n"
                             f"    error:      {error_eu_float}")

        error_am_float = abs(am_floating_convertible.NPV() - vanilla_floating_bond.settlementValue())
        self.assertLessEqual(error_am_float, tolerance_float,
                             f"Failed to reproduce AM floating-rate bond price:\n"
                             f"    calculated: {am_floating_convertible.NPV()}\n"
                             f"    expected:   {vanilla_floating_bond.settlementValue()}\n"
                             f"    error:      {error_am_float}")


    def test_option(self):
        """
        A zero-coupon convertible bond with no credit spread is
        equivalent to a call option.
        """
        print("Testing zero-coupon convertible bonds against vanilla option...")

        eu_exercise = ql.EuropeanExercise(self.maturity_date)

        original_settlement_days = self.settlement_days
        self.settlement_days = 0 # As per C++ test

        time_steps_option = 2001

        # Temporarily set credit spread to zero for this test
        original_credit_spread = self.credit_spread_quote.value()
        self.credit_spread_quote.setValue(0.0) # This updates the handle
        # Recreate engine with zero spread (or ensure handle update propagates)
        convertible_engine_option_test = ql.BinomialConvertibleEngine(
            self.process, "CoxRossRubinstein", time_steps_option, self.credit_spread_handle
        )

        vanilla_engine_option_test = ql.BinomialVanillaEngine(
            self.process, "CoxRossRubinstein", time_steps_option
        )

        # Use the default conversion ratio from setUp
        conversion_ratio_test = self.conversion_ratio_default
        conversion_strike = self.redemption / conversion_ratio_test
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, conversion_strike)

        schedule_option_test = ql.MakeSchedule().from_(self.issue_date).to(self.maturity_date) \
                                           .withFrequency(ql.Once) \
                                           .withCalendar(self.calendar) \
                                           .backwards().makeSchedule()

        eu_zero_convertible_for_option_test = ql.ConvertibleZeroCouponBond(
            eu_exercise, conversion_ratio_test, self.no_callability,
            self.issue_date, self.settlement_days, self.day_counter,
            schedule_option_test, self.redemption
        )
        eu_zero_convertible_for_option_test.setPricingEngine(convertible_engine_option_test)

        eu_vanilla_option = ql.VanillaOption(payoff, eu_exercise)
        eu_vanilla_option.setPricingEngine(vanilla_engine_option_test)

        tolerance_option = 5.0e-2 * (self.face_amount / 100.0)

        expected_value = (self.face_amount / 100.0) * \
                         (self.redemption * self.risk_free_rate_handle.discount(self.maturity_date) +
                          conversion_ratio_test * eu_vanilla_option.NPV())

        calculated_npv = eu_zero_convertible_for_option_test.NPV()
        error_option = abs(calculated_npv - expected_value)

        self.assertLessEqual(error_option, tolerance_option,
                             f"Failed to reproduce plain-option price:\n"
                             f"    calculated: {calculated_npv}\n"
                             f"    expected:   {expected_value}\n"
                             f"    error:      {error_option}\n"
                             f"    tolerance:  {tolerance_option}")

        # Restore original values
        self.credit_spread_quote.setValue(original_credit_spread)
        self.settlement_days = original_settlement_days


    def test_regression(self):
        """Testing fixed-coupon convertible bond in known regression case."""
        print("Testing fixed-coupon convertible bond in known regression case...")

        saved_eval_date = ql.Settings.instance().evaluationDate

        today_reg = ql.Date(23, ql.December, 2008)
        tomorrow_reg = today_reg + 1
        ql.Settings.instance().evaluationDate = tomorrow_reg

        underlying_quote_reg = ql.SimpleQuote(2.9084382818797443)
        underlying_handle_reg = ql.QuoteHandle(underlying_quote_reg)

        dates_reg_data = [
            (ql.Date(29,ql.December,2008), 0.0025999342800), (ql.Date(5,ql.January,2009), 0.0025999342800),
            (ql.Date(29,ql.January,2009), 0.0053123275500), (ql.Date(27,ql.February,2009), 0.0197049598721),
            (ql.Date(30,ql.March,2009), 0.0220524845296), (ql.Date(29,ql.June,2009), 0.0217076395643),
            (ql.Date(29,ql.December,2009), 0.0230349627478), (ql.Date(29,ql.December,2010), 0.0087631647476),
            (ql.Date(29,ql.December,2011), 0.0219084299499), (ql.Date(31,ql.December,2012), 0.0244798766219),
            (ql.Date(30,ql.December,2013), 0.0267885498456), (ql.Date(29,ql.December,2014), 0.0266922867562),
            (ql.Date(29,ql.December,2015), 0.0271052126386), (ql.Date(29,ql.December,2016), 0.0268829891648),
            (ql.Date(29,ql.December,2017), 0.0264594744498), (ql.Date(31,ql.December,2018), 0.0273450367424),
            (ql.Date(30,ql.December,2019), 0.0294852614749), (ql.Date(29,ql.December,2020), 0.0285556119719),
            (ql.Date(29,ql.December,2021), 0.0305557764659), (ql.Date(29,ql.December,2022), 0.0292244738422),
            (ql.Date(29,ql.December,2023), 0.0263917004194), (ql.Date(29,ql.December,2028), 0.0239626970243),
            (ql.Date(29,ql.December,2033), 0.0216417108090), (ql.Date(29,ql.December,2038), 0.0228343838422),
            (ql.Date(31,ql.December,2199), 0.0228343838422)
        ]
        dates_reg = [d[0] for d in dates_reg_data]
        forwards_reg = [d[1] for d in dates_reg_data]

        r_handle_reg = ql.YieldTermStructureHandle(
            ql.ForwardCurve(dates_reg, forwards_reg, ql.Actual360())
        )
        sigma_handle_reg = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(tomorrow_reg, ql.NullCalendar(),
                                ql.QuoteHandle(ql.SimpleQuote(21.685235548092248 / 100.0)), # Vol is % in C++
                                ql.Thirty360(ql.Thirty360.BondBasis))
        )
        process_reg = ql.BlackProcess(underlying_handle_reg, r_handle_reg, sigma_handle_reg)

        spread_quote_reg = ql.SimpleQuote(0.11498700678012874)
        spread_handle_reg = ql.QuoteHandle(spread_quote_reg)

        issue_date_reg = ql.Date(23, ql.July, 2008)
        maturity_date_reg = ql.Date(1, ql.August, 2013)
        calendar_reg = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
        schedule_reg = ql.MakeSchedule().from_(issue_date_reg).to(maturity_date_reg) \
                                     .withTenor(ql.Period(6, ql.Months)) \
                                     .withCalendar(calendar_reg) \
                                     .withConvention(ql.Unadjusted).makeSchedule()
        settlement_days_reg = 3
        exercise_reg = ql.EuropeanExercise(maturity_date_reg)
        conversion_ratio_reg = 100.0 / 20.3175
        coupons_reg = [0.05] * (len(schedule_reg) -1) # size is num of periods
        day_counter_reg = ql.Thirty360(ql.Thirty360.BondBasis)
        no_callability_reg = ql.CallabilitySchedule()
        no_dividends_reg = ql.DividendSchedule() # Empty implies no dividends for engine
        redemption_reg = 100.0

        bond_reg = ql.ConvertibleFixedCouponBond(
            exercise_reg, conversion_ratio_reg, no_callability_reg,
            issue_date_reg, settlement_days_reg, coupons_reg, day_counter_reg,
            schedule_reg, redemption_reg
        )

        bond_reg.setPricingEngine(
            ql.BinomialConvertibleEngine(process_reg, "CoxRossRubinstein", 600, spread_handle_reg, no_dividends_reg)
        )

        try:
            x = bond_reg.NPV()
            # If NPV() doesn't throw, then an INF/NaN was not detected as an error.
            self.fail(f"INF result was not detected: {x} returned")
        except ql.Error as e:
            # Expected to throw QuantLib::Error due to INF/NaN in pricing
            print(f"Regression test: Caught expected QuantLib Error: {e}")
            pass # Test passes if ql.Error is caught
        except Exception as e:
            self.fail(f"Regression test: Caught unexpected exception: {type(e).__name__} - {e}")
        finally:
            # Restore original evaluation date
            ql.Settings.instance().evaluationDate = saved_eval_date


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    # Set evaluation date if required, TopLevelFixture usually handles this.
    # For these tests, setUp method in the class takes care of setting self.today
    # from ql.Settings.instance().evaluationDate
    # eval_date = ql.Date(15, ql.May, 1998) # Example
    # ql.Settings.instance().evaluationDate = eval_date
    unittest.main(argv=['first-arg-is-ignored'], exit=False)